In [1]:
import sys; sys.path.insert(0, '..') 

In [2]:
import numpy as np

In [3]:
import numpy as np
import torch
from preprocess.dataset import get_MNIST,get_dataset,get_handler
from models.model import Model
from al_methods.least_confidence import LeastConfidence
from ssl_methods.semi_fixmatch import fixmatch
import models
from torchvision import transforms
from framework.framework1 import Framework1,Strategy1,Strategy2
from copy import deepcopy
import torch.nn as nn
import time

In [4]:
# in the main 
X_tr, Y_tr, X_te, Y_te = get_dataset("Mnist", "./datasets")

100%|██████████| 9912422/9912422 [00:02<00:00, 3897489.32it/s]


Extracting ./datasets/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./datasets/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 109724360.35it/s]


Extracting ./datasets/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./datasets/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 20405554.66it/s]


Extracting ./datasets/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./datasets/mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 12126370.95it/s]

Extracting ./datasets/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./datasets/mnist/MNIST/raw



In [5]:
# in the main 
if type(X_tr) is list:
    X_tr = np.array(X_tr)
    Y_tr = torch.tensor(np.array(Y_tr))
    X_te = np.array(X_te)
    Y_te = torch.tensor(np.array(Y_te))

if type(X_tr[0]) is not np.ndarray:
    X_tr = X_tr.numpy()
    X_te = X_te.numpy()
    

In [6]:
np.shape(X_tr)[1:]

(28, 28)

In [7]:
# in the main
n_pool = len(Y_tr)
n_test = len(Y_te)

In [8]:
#in the main
handler = get_handler("mnist")

In [9]:
# main or framewrok to see
nEnd=100 # total number to query 
nQuery=1 # nombre of points to query in batch 
nStart=1 # nbre of points to start
NUM_INIT_LB = int(nStart*n_pool/100)
NUM_QUERY = int(nQuery*n_pool/100) if nStart!= 100 else 0
NUM_ROUND = int((int(nEnd*n_pool/100) - NUM_INIT_LB)/ NUM_QUERY) if nStart!= 100 else 0
if NUM_QUERY != 0:
    if (int(nEnd*n_pool/100) - NUM_INIT_LB)% NUM_QUERY != 0:
        NUM_ROUND += 1

In [10]:
print(NUM_INIT_LB,NUM_ROUND,NUM_QUERY)

600 99 600


In [11]:
model_name = sorted(name for name in models.__dict__ 
                    if callable(models.__dict__[name]))

In [12]:
model_name

[]

In [13]:
model=Model('resnet50').get_model()

In [14]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [15]:
# net = models.__dict__['ResNet18'](n_class=10)

In [16]:
# net

In [17]:
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # Conversion en 3 canaux
model.fc = torch.nn.Linear(2048, 10)  # Modifier la couche de classification pour 10 classes

In [18]:
# net.feature_extractor.conv1=torch.nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [19]:
# net

In [20]:
# in the main file
idxs_lb = np.zeros(n_pool, dtype=bool)
idxs_lb

array([False, False, False, ..., False, False, False])

In [21]:
# in the main file
idxs_tmp = np.arange(n_pool)
idxs_tmp

array([    0,     1,     2, ..., 59997, 59998, 59999])

In [22]:
np.random.shuffle(idxs_tmp)
idxs_tmp

array([41585, 22843, 27165, ..., 48810, 14362, 30444])

In [23]:
# in the main file
idxs_lb[idxs_tmp[:NUM_INIT_LB]] = True
idxs_lb

array([False, False, False, ..., False, False, False])

### Args

in the main file 

In [24]:
args_pool = {'mnist':
                { 
                 'n_class':10,
                 'channels':1,
                 'size': 28,
                 'transform_tr': transforms.Compose([
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'transform_te': transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))]),
                 'loader_tr_args':{'batch_size': 128, 'num_workers': 8},
                 'loader_te_args':{'batch_size': 1024, 'num_workers': 8},
                 'normalize':{'mean': (0.1307,), 'std': (0.3081,)},
                },
            
        }

In [25]:
class Args:
    def __init__(self,n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize):
        self.n_class=n_class
        self.img_size=img_size
        self.channels=channels
        self.transform_tr=transform_tr
        self.transform_te=transform_te
        self.loader_tr_args=loader_tr_args
        self.loader_te_args=loader_te_args
        self.normalize=normalize
        self.dataset='mnist'
        self.save_path='./save'
        self.model='ResNet50'
        self.lr=0.1
        self.schedule = [20, 40]
        self.momentum=0.9
        self.gammas=[0.1,0.1]
        self.strategy='framwork1'
        self.optimizer='SGD'
        self.save_model=False

In [26]:
dataset_args = args_pool["mnist"]
n_class = dataset_args['n_class']
img_size = dataset_args['size']
channels = dataset_args['channels']
transform_tr = dataset_args['transform_tr']
transform_te = dataset_args['transform_te']
loader_tr_args = dataset_args['loader_tr_args']
loader_te_args = dataset_args['loader_te_args']
normalize = dataset_args['normalize']

In [27]:
args=Args(n_class,img_size,channels,transform_tr,transform_te,loader_tr_args,loader_te_args,normalize)

### Framework setting

In [28]:
# Create instances of strategies
random_strategy = LeastConfidence(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args)
fixmatch_strategy = fixmatch(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args)

TypeError: LeastConfidence.__init__() missing 2 required positional arguments: 'n_pool' and 'device'

In [ ]:
# Create an instance of Framework1
framework_1 = Framework1(X_tr, Y_tr, X_te, Y_te, idxs_lb, model, handler, args, random_strategy, fixmatch_strategy)

In [ ]:
al_method.train(2e-3,10)

In [ ]:
test_acc=al_method.predict(X_te,Y_te)

In [ ]:
test_acc

In [ ]:
acc = np.zeros(NUM_ROUND+1)
acc[0] = test_acc

In [ ]:
for rd in range(1, NUM_ROUND+1):

    if rd%2==0:
        # Al_methods
        print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
        labeled = len(np.arange(n_pool)[idxs_lb])
        if NUM_QUERY > int(nEnd*n_pool/100) - labeled:
            NUM_QUERY = int(nEnd*n_pool/100) - labeled
            
        # query
        ts = time.time()
        output = framework2.stratAl.query(NUM_QUERY)
        q_idxs = output
        idxs_lb[q_idxs] = True
        te = time.time()
        tp = te - ts
        
        # update
        al_method.update(idxs_lb)
        best_test_acc = framework2.train(alpha=2e-3, n_epoch=10)

        t_iter = time.time() - ts
        
        # round accuracy
        # test_acc = strategy.predict(X_te, Y_te)
        acc[rd] = best_test_acc
    else:
        #SSL methods
        
        print('Round {}/{}'.format(rd, NUM_ROUND), flush=True)
        labeled = len(np.arange(n_pool)[idxs_lb])
        if NUM_QUERY > int(nEnd*n_pool/100) - labeled:
            NUM_QUERY = int(nEnd*n_pool/100) - labeled
            
        # query
        ts = time.time()
        output = ssl_method.query(NUM_QUERY)
        q_idxs = output
        idxs_lb[q_idxs] = True
        te = time.time()
        tp = te - ts
        
        # update
        ssl_method.update(idxs_lb)
        best_test_acc = framework.stratSSL.train(alpha=2e-3, n_epoch=10)

        t_iter = time.time() - ts
        
        # round accuracy
        # test_acc = strategy.predict(X_te, Y_te)
        acc[rd] = best_test_acc


In [ ]:
acc